# Import



In [ ]:
import os
import os.path as osp
import time
import datetime
import random
from PIL import Image
import numpy as np
from torch.autograd import Variable
from sklearn.metrics import accuracy_score
import torch.nn as nn
from torch.autograd import Function
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
import tensorflow as tf
import torch
import torchvision
import matplotlib.pyplot as plt
import pandas as pd
import torch.utils.data as data
from torch.utils.data import random_split, TensorDataset, DataLoader, RandomSampler, SequentialSampler, WeightedRandomSampler
import logging
from torchvision import datasets,transforms, models
import tqdm
import glob
import cv2
import tensorflow_hub as hub
%matplotlib inline
use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")

In [ ]:
print("Version ", tf.__version__)
print("Eager mode:", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is","available" if tf.test.is_gpu_available() else"Not Available")

# Prepare Data

## Tomato

In [ ]:
source_tomato = '/kaggle/input/plant-village/PlantVillage'
os.listdir(source_tomato)

In [ ]:
import shutil
source_folder= source_tomato
destination_folder=os.path.join("/kaggle/working/","Tomato_dataset")
if not os.path.exists(destination_folder):
    os.mkdir(destination_folder)

    subdirectories = os.listdir(source_folder)
    for subdirectory in subdirectories:
        if (subdirectory[:6]=='Tomato'):
            subdirectory_path = os.path.join(source_folder, subdirectory)
            shutil.copytree(subdirectory_path, os.path.join(destination_folder, subdirectory))

## Remove File

In [ ]:
files2 = [file for file in os.listdir('/kaggle/working/cornCommon_Rust')]

In [ ]:
def remove_folder_contents(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                remove_folder_contents(file_path)
                os.rmdir(file_path)
        except Exception as e:
            print(e)

folder_path = '/kaggle/working/Potato_dataset'
remove_folder_contents(folder_path)
os.rmdir(folder_path)

## Potato

In [ ]:
source_potato = '/kaggle/input/potato-disease-leaf-datasetpld/PLD_3_Classes_256'
os.listdir(source_potato)

In [ ]:
source_folder = source_potato
destination_folder = os.path.join("/kaggle/working/","Potato_dataset")
if not os.path.exists(destination_folder):
    os.mkdir(destination_folder)

    subdirectories = os.listdir(source_folder)
    for subdirectory in subdirectories:
        subdirectory_path = os.path.join(source_folder, subdirectory)
        subs = os.listdir(subdirectory_path)
        for sub in subs:
#             print(sub)
            sub_path = os.path.join(subdirectory_path, sub)
            sub = 'Potato_' + sub
            destination = os.path.join(destination_folder, subdirectory)
#             print(destination)
            shutil.copytree(sub_path, os.path.join(destination, sub))
#         subdirectory = 'Corn_' + subdirectory
#         shutil.copytree(subdirectory_path, os.path.join(destination_folder, subdirectory))

## Corn

In [ ]:
source_corn = '/kaggle/input/corn-or-maize-leaf-disease-dataset/data'
os.listdir(source_corn)

In [ ]:
import shutil
source_folder= source_corn
destination_folder=os.path.join("/kaggle/working/","Corn_dataset")
if not os.path.exists(destination_folder):
    os.mkdir(destination_folder)

    subdirectories = os.listdir(source_folder)
    for subdirectory in subdirectories:
        subdirectory_path = os.path.join(source_folder, subdirectory)
        subdirectory = 'Corn_' + subdirectory
        shutil.copytree(subdirectory_path, os.path.join(destination_folder, subdirectory))

## RICE

In [ ]:
source_rice = '/kaggle/input/rice-diseases-image-dataset/LabelledRice/Labelled'
os.listdir(source_rice)

In [ ]:
import shutil
source_folder= source_rice
destination_folder=os.path.join("/kaggle/working/","Rice_dataset")
if not os.path.exists(destination_folder):
    os.mkdir(destination_folder)

    subdirectories = os.listdir(source_folder)
    for subdirectory in subdirectories:
        subdirectory_path = os.path.join(source_folder, subdirectory)
        subdirectory = 'Rice_' + subdirectory
        shutil.copytree(subdirectory_path, os.path.join(destination_folder, subdirectory))

In [ ]:
l = []
for i in data:
    l.append(labels[i[1]])
sns.set_style('dark')
sns.countplot(l)

## Transform

In [ ]:
batch_size = 8
image_size = (224,224)
channels = 3

In [ ]:
data_corn = tf.keras.utils.image_dataset_from_directory(
    directory = '/kaggle/working/Corn_dataset',
    color_mode ='rgb',
    batch_size = batch_size,
    image_size = image_size,
    shuffle = True
)
data_tomato = tf.keras.utils.image_dataset_from_directory(
    directory = '/kaggle/working/Tomato_dataset',
    color_mode ='rgb',
    batch_size = batch_size,
    image_size = image_size,
    shuffle = True
)

In [ ]:
dirs_in=data_corn.class_names
dirs_in

In [ ]:
len(data_corn)

In [ ]:
for batch,labels in data_corn.take(1):
    print(f"Shape of per batch: {batch.shape} Type: {type(batch)}")
    print(f"Shape of labels: {labels.shape} Type: {type(labels)}")

## Split Data

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder

def split_dataset(dataset, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1, batch_size=8, shuffle=True):
    num_samples = len(dataset)
    num_train = int(train_ratio * num_samples)
    num_val = int(val_ratio * num_samples)
    num_test = num_samples - num_train - num_val
    
    train_data, val_data, test_data = random_split(dataset, [num_train, num_val, num_test])
    
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=shuffle)
    val_loader = DataLoader(val_data, batch_size=batch_size)
    test_loader = DataLoader(test_data, batch_size=batch_size)
    
    return train_loader, val_loader, test_loader


# Use the split_dataset function
train_dataloader_corn, val_dataloader_corn, test_dataloader_corn = split_dataset(data_corn)
train_dataloader_tomato, val_dataloader_tomato, test_dataloader_tomato = split_dataset(data_tomato)

In [ ]:
len(val_dataloader_corn)

In [ ]:
data_transforms = {...}
data_dir = "path_to_your_data_directory"

# Create the original ImageFolder dataset
original_dataset = ImageFolder(root=data_dir, transform=data_transforms['train'])

# Assuming you want to convert a subset of the dataset into an ImageFolder dataset
subset_indices = [0, 1, 2, 3, ...]  # Replace with your desired subset indices
subset_dataset = ImageFolder(root=data_dir, transform=data_transforms['train'], loader=lambda x: original_dataset.loader(x))
subset_dataset = Subset(subset_dataset, subset_indices)


In [ ]:
def split_dataset(dataset,train_size=0.8,test_size=0.1,val_size=0.1,shuffle=True,shuffle_size=10000):
    if shuffle:
        dataset=dataset.shuffle(shuffle_size,seed=20)
    train_data=dataset.take(int(train_size*len(dataset)))
    test_data=dataset.skip(int(train_size*len(dataset))).take(int(test_size*len(dataset)))
    val_data=dataset.skip(int((train_size+test_size)*len(dataset)))
    
    return train_data,test_data,val_data

In [ ]:
corn_train,corn_test,corn_val = split_dataset(data_corn)
len(corn_train),len(corn_test),len(corn_val)

In [ ]:
tomato_train,tomato_test,tomato_val = split_dataset(data_tomato)
len(tomato_train),len(tomato_test),len(tomato_val)

In [ ]:
corn_train=corn_train.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
corn_test=corn_test.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
corn_val=corn_val.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
tomato_train = tomato_train.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
tomato_test = tomato_test.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
tomato_val = tomato_val.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
len(tomato_train)

In [ ]:
type(corn_train)

In [ ]:
fetch_to_if(corn_train)

In [ ]:
#Split data RICE
import os
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToPILImage
from torch.utils.data import random_split

# Load your existing ImageFolder dataset
imagefolder_tomato = ImageFolder(root="/kaggle/working/Rice_dataset", transform=data_transforms[TRAIN])

# Split the dataset
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

num_samples = len(imagefolder_tomato)
num_train = int(train_ratio * num_samples)
num_val = int(val_ratio * num_samples)
num_test = num_samples - num_train - num_val

train_data, val_data, test_data = random_split(imagefolder_tomato, [num_train, num_val, num_test])

# Create output directories
output_dir = "/kaggle/working/Potato_dataset"
train_output_dir = os.path.join(output_dir, "Training")
val_output_dir = os.path.join(output_dir, "Validation")
test_output_dir = os.path.join(output_dir, "Testing")

os.makedirs(train_output_dir, exist_ok=True)
os.makedirs(val_output_dir, exist_ok=True)
os.makedirs(test_output_dir, exist_ok=True)

# Save images to output directories
to_pil = ToPILImage()
i = 1
for data, label in train_data:
    class_name =imagefolder_tomato.classes[label]
    class_output_dir = os.path.join(train_output_dir, class_name)
    os.makedirs(class_output_dir, exist_ok=True)
    image = to_pil(data)
    image.save(os.path.join(class_output_dir, f"image_{i}.jpg"))
    i += 1
print(i)
i = 1
for data, label in val_data:
    class_name =imagefolder_tomato.classes[label]
    class_output_dir = os.path.join(val_output_dir, class_name)
    os.makedirs(class_output_dir, exist_ok=True)
    image = to_pil(data)
    image.save(os.path.join(class_output_dir, f"image_{i}.jpg"))
    i += 1
print(i)
i = 1
for data, label in test_data:
    class_name =imagefolder_tomato.classes[label]
    class_output_dir = os.path.join(test_output_dir, class_name)
    os.makedirs(class_output_dir, exist_ok=True)
    image = to_pil(data)
    image.save(os.path.join(class_output_dir, f"image_{i}.jpg"))
    i += 1
print(i)

In [ ]:
!zip -r file.zip /kaggle/working/Tomato_dataset

In [ ]:
import os
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToPILImage
from torch.utils.data import random_split

# Load your existing ImageFolder dataset
imagefolder_tomato = ImageFolder(root="/kaggle/working/Tomato_dataset", transform=data_transforms[TRAIN])

# Split the dataset
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

num_samples = len(imagefolder_tomato)
num_train = int(train_ratio * num_samples)
num_val = int(val_ratio * num_samples)
num_test = num_samples - num_train - num_val

train_data, val_data, test_data = random_split(imagefolder_tomato, [num_train, num_val, num_test])

# Create output directories
output_dir = "/kaggle/working/Potato_dataset"
train_output_dir = os.path.join(output_dir, "Training")
val_output_dir = os.path.join(output_dir, "Validation")
test_output_dir = os.path.join(output_dir, "Testing")

os.makedirs(train_output_dir, exist_ok=True)
os.makedirs(val_output_dir, exist_ok=True)
os.makedirs(test_output_dir, exist_ok=True)

# Save images to output directories
to_pil = ToPILImage()
i = 1
for data, label in train_data:
    class_name =imagefolder_tomato.classes[label]
    class_output_dir = os.path.join(train_output_dir, class_name)
    os.makedirs(class_output_dir, exist_ok=True)
    image = to_pil(data)
    image.save(os.path.join(class_output_dir, f"image_{i}.jpg"))
    i += 1
print(i)
i = 1
for data, label in val_data:
    class_name =imagefolder_tomato.classes[label]
    class_output_dir = os.path.join(val_output_dir, class_name)
    os.makedirs(class_output_dir, exist_ok=True)
    image = to_pil(data)
    image.save(os.path.join(class_output_dir, f"image_{i}.jpg"))
    i += 1
print(i)
i = 1
for data, label in test_data:
    class_name =imagefolder_tomato.classes[label]
    class_output_dir = os.path.join(test_output_dir, class_name)
    os.makedirs(class_output_dir, exist_ok=True)
    image = to_pil(data)
    image.save(os.path.join(class_output_dir, f"image_{i}.jpg"))
    i += 1
print(i)

In [ ]:
import os
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToPILImage
from torch.utils.data import random_split

# Load your existing ImageFolder dataset
imagefolder_tomato = ImageFolder(root="/kaggle/working/Corn_dataset", transform=data_transforms[TRAIN])

# Split the dataset
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

num_samples = len(imagefolder_tomato)
num_train = int(train_ratio * num_samples)
num_val = int(val_ratio * num_samples)
num_test = num_samples - num_train - num_val

train_data, val_data, test_data = random_split(imagefolder_tomato, [num_train, num_val, num_test])

# Create output directories
output_dir = "/kaggle/working/Potato_dataset"
train_output_dir = os.path.join(output_dir, "Training")
val_output_dir = os.path.join(output_dir, "Validation")
test_output_dir = os.path.join(output_dir, "Testing")

os.makedirs(train_output_dir, exist_ok=True)
os.makedirs(val_output_dir, exist_ok=True)
os.makedirs(test_output_dir, exist_ok=True)

# Save images to output directories
to_pil = ToPILImage()
i = 1
for data, label in train_data:
    class_name =imagefolder_tomato.classes[label]
    class_output_dir = os.path.join(train_output_dir, class_name)
    os.makedirs(class_output_dir, exist_ok=True)
    image = to_pil(data)
    image.save(os.path.join(class_output_dir, f"image_{i}.jpg"))
    i += 1
print(i)
i = 1
for data, label in val_data:
    class_name =imagefolder_tomato.classes[label]
    class_output_dir = os.path.join(val_output_dir, class_name)
    os.makedirs(class_output_dir, exist_ok=True)
    image = to_pil(data)
    image.save(os.path.join(class_output_dir, f"image_{i}.jpg"))
    i += 1
print(i)
i = 1
for data, label in test_data:
    class_name =imagefolder_tomato.classes[label]
    class_output_dir = os.path.join(test_output_dir, class_name)
    os.makedirs(class_output_dir, exist_ok=True)
    image = to_pil(data)
    image.save(os.path.join(class_output_dir, f"image_{i}.jpg"))
    i += 1
print(i)

In [ ]:
folder_path = "/kaggle/working/Tomato_Folder/Validation/Tomato_Early_blight"

# List the files in the folder
file_list = os.listdir(folder_path)

# Calculate the number of files
num_files = len(file_list)
print(num_files)

In [ ]:
len('/kaggle/working/Tomato_Folder/Training')

In [ ]:
file.remove("/kaggle/working/file.zip")

In [ ]:
data_dir = '/kaggle/input/potato-disease-leaf-datasetpld/PLD_3_Classes_256'
TRAIN = 'Training'
VAL = 'Validation'
TEST = 'Testing'

# VGG-16 Takes 224x224 images as input, so we resize all of them
data_transforms = {
    TRAIN: transforms.Compose([
        # Data augmentation is a good practice for the train set
        # Here, we randomly crop the image to 224x224 and
        # randomly flip it horizontally. 
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    VAL: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ]),
    TEST: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
}

In [ ]:
from torch.utils.data import ConcatDataset
data_dir = '/kaggle/working/Potato_dataset'
TRAIN = 'Training'
VAL = 'Validation'
TEST = 'Testing'

# VGG-16 Takes 224x224 images as input, so we resize all of them
data_transforms = {
    TRAIN: transforms.Compose([
        # Data augmentation is a good practice for the train set
        # Here, we randomly crop the image to 224x224 and
        # randomly flip it horizontally. 
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    VAL: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ]),
    TEST: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
}

image_datasets = {
    x: datasets.ImageFolder(
        os.path.join(data_dir, x), 
        transform=data_transforms[x]
    )
    for x in [TRAIN, VAL, TEST]
}
# new_class_names = ['Potato_Early_Blight', 'Potato_Healthy', 'Potato_Late_Blight']
# for x in [TRAIN, VAL, TEST]:
#     image_datasets[x].classes = new_class_names
# type(image_datasets[TRAIN])
# corn_dir = '/kaggle/working/Corn_Folder'
# tomato_dir = '/kaggle/working/Tomato_Folder'
# corn_datasets = {
#     x: datasets.ImageFolder(
#         os.path.join(corn_dir, x), 
#         transform=data_transforms[x]
#     )
#     for x in [TRAIN, VAL, TEST]
# }
# tomato_datasets = {
#     x: datasets.ImageFolder(
#         os.path.join(tomato_dir, x), 
#         transform=data_transforms[x]
#     )
#     for x in [TRAIN, VAL, TEST]
# }
# image_datasets[TRAIN] += corn_datasets[TRAIN]
# image_datasets[TRAIN] = ConcatDataset([image_datasets[TRAIN], corn_datasets[TRAIN]])

# image_datasets[TRAIN] += corn_datasets[TRAIN]
# image_datasets[TRAIN] += tomato_datasets[TRAIN]
# print(type(image_datasets))
# image_datasets[TRAIN] += corn_train
# type(image_datasets[TRAIN])
# class_names = image_datasets[TRAIN].classes
# len(class_names)  
dataloaders = {
    x: torch.utils.data.DataLoader(
        image_datasets[x], batch_size=8,
        shuffle=True, num_workers=4
    )
    for x in [TRAIN, VAL, TEST]
}
dataset_sizes = {x: len(image_datasets[x]) for x in [TRAIN, VAL, TEST]}

for x in [TRAIN, VAL, TEST]:
    print("Loaded {} images under {}".format(dataset_sizes[x], x))

class_names = image_datasets[TRAIN].classes

print(image_datasets[TEST].classes)   

In [ ]:
len(class_names)

In [ ]:
data_transforms = {
    TRAIN: transforms.Compose([
        # Data augmentation is a good practice for the train set
        # Here, we randomly crop the image to 224x224 and
        # randomly flip it horizontally. 
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(0.1),
#         transforms.ToTensor(),
        transforms.Normalize(mean=[0.4884, 0.4551, 0.4170], std=[0.2256, 0.2210, 0.2214]),
#         transforms.ToTensor(),
    ]),
    VAL: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.Normalize(mean=[0.4884, 0.4551, 0.4170], std=[0.2256, 0.2210, 0.2214]),
        transforms.ToTensor(),
    ]),
    TEST: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
}

In [ ]:
data_transforms = {
    TRAIN: transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(0.1),
        transforms.ToTensor(),  # Convert PIL Image to Tensor
        transforms.Normalize(mean=[0.4884, 0.4551, 0.4170], std=[0.2256, 0.2210, 0.2214]),
    ]),
    VAL: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),  # Convert PIL Image to Tensor
        transforms.Normalize(mean=[0.4884, 0.4551, 0.4170], std=[0.2256, 0.2210, 0.2214]),
    ]),
    TEST: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),  # Convert PIL Image to Tensor
    ])
}


In [ ]:
# for rice only
from torch.utils.data import ConcatDataset
data_dir = '/kaggle/working/Rice_folder'
TRAIN = 'Training'
VAL = 'Validation'
TEST = 'Testing'

# VGG-16 Takes 224x224 images as input, so we resize all of them
# data_transforms = {
#     TRAIN: transforms.Compose([
#         # Data augmentation is a good practice for the train set
#         # Here, we randomly crop the image to 224x224 and
#         # randomly flip it horizontally. 
#         transforms.RandomResizedCrop(224),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#     ]),
#     VAL: transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#     ]),
#     TEST: transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#     ])
# }

image_datasets = {
    x: datasets.ImageFolder(
        os.path.join(data_dir, x), 
        transform=data_transforms[x]
    )
    for x in [TRAIN, VAL, TEST]
}
dataloaders = {
    x: torch.utils.data.DataLoader(
        image_datasets[x], batch_size=8,
        shuffle=True, num_workers=4
    )
    for x in [TRAIN, VAL, TEST]
}
dataset_sizes = {x: len(image_datasets[x]) for x in [TRAIN, VAL, TEST]}

for x in [TRAIN, VAL, TEST]:
    print("Loaded {} images under {}".format(dataset_sizes[x], x))

class_names = image_datasets[TRAIN].classes

print(image_datasets[TEST].classes)   

In [ ]:
print(class_names[1])

In [ ]:
dataloaders[TRAIN] = DataLoader(
    ConcatDataset([corn_data, ]),
    batch_size=batch_size,
    shuffle=True
)

In [ ]:
# def imshow(inp, title=None):
#     inp = inp.numpy().transpose((1, 2, 0))
#     plt.figure(figsize=(10, 10))
#     plt.axis('off')
#     plt.imshow(inp)
#     if title is not None:
#         plt.title(title)
#     plt.pause(0.001)

# def show_databatch(inputs, classes):
#     out = torchvision.utils.make_grid(inputs)
#     imshow(out, title=[class_names[x] for x in classes])

# # Get a batch of training data
# inputs, classes = next(iter(dataloaders[TRAIN]))
# show_databatch(inputs, classes)

In [ ]:
class_name = class_names

In [ ]:
class_names

# Visualization

In [ ]:
def imshow_with_title(inp, title):
    inp = inp.numpy().transpose((1, 2, 0))
    plt.figure()
    plt.imshow(inp)
    plt.title(title)
    plt.axis('off')
    plt.show()

def show_databatch_with_titles(inputs, class_names, cols=4):
    batch_size, num_channels, height, width = inputs.shape
    rows = int(np.ceil(batch_size / cols))
    fig, axes = plt.subplots(rows, cols, figsize=(9, 9))
    for ax in axes.flatten():
        ax.axis('off')
    
    for i in range(batch_size):
        row = i // cols
        col = i % cols
        ax = axes[row,col]
        img = inputs[i].numpy().transpose((1, 2, 0))
        ax.imshow(img)
#         print(class_names[i].item())
        ax.set_title(class_name[class_names[i].item()], fontsize = 7.5)
#         print(class_names[i])
    
    plt.tight_layout()
    plt.show()

inputs, classes = next(iter(dataloaders[TRAIN]))

show_databatch_with_titles(inputs, classes)

In [ ]:
def visualize_model(vgg, num_images=6):
    was_training = vgg.training
    
    # Set model for evaluation
    vgg.train(False)
    vgg.eval() 
    
    images_so_far = 0

    for i, data in enumerate(dataloaders[TEST]):
        inputs, labels = data
        size = inputs.size()[0]
        inputs, labels = inputs.cuda(), labels.cuda()
#         if use_gpu:
#             inputs, labels = Variable(inputs.cuda(), volatile=True), Variable(labels.cuda(), volatile=True)
#         else:
#             inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)
        
        outputs = vgg(inputs)
        
        _, preds = torch.max(outputs.data, 1)
        predicted_labels = [preds[j] for j in range(inputs.size()[0])]
        
        print("Ground truth:")
        show_databatch_with_titles(inputs.data.cpu(), labels.data.cpu())
        print("Prediction:")
        show_databatch_with_titles(inputs.data.cpu(), predicted_labels)
        
        del inputs, labels, outputs, preds, predicted_labels
        torch.cuda.empty_cache()
        
        images_so_far += size
        if images_so_far >= num_images:
            break
        
    vgg.train(mode=was_training) # Revert model back to original training state

# Testing

In [ ]:
def eval_model(vgg, criterion):
    since = time.time()
    avg_loss = 0
    avg_acc = 0
    loss_test = 0
    acc_test = 0
    
    test_batches = len(dataloaders[TEST])
    print("Evaluating model")
    print('-' * 10)
    
    for i, data in enumerate(dataloaders[TEST]):
        if i % 100 == 0:
            print("\rTest batch {}/{}".format(i, test_batches), end='', flush=True)

        vgg.train(False)
        vgg.eval()
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
#         if use_gpu:
#             inputs, labels = Variable(inputs.cuda(), volatile=True), Variable(labels.cuda(), volatile=True)
#         else:
#             inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)

        outputs = vgg(inputs)

        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)

        loss_test += loss.data
        acc_test += torch.sum(preds == labels.data)

        del inputs, labels, outputs, preds
        torch.cuda.empty_cache()
        
    avg_loss = loss_test / dataset_sizes[TEST]
    avg_acc = acc_test / dataset_sizes[TEST]
    
    elapsed_time = time.time() - since
    print()
    print("Evaluation completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Avg loss (test): {:.4f}".format(avg_loss))
    print("Avg acc (test): {:.4f}".format(avg_acc))
    print('-' * 10)

# Load Model

In [ ]:
# Load the pretrained model from pytorch
vgg16 = models.vgg16_bn()
vgg16.load_state_dict(torch.load("/kaggle/input/vgg16bn/vgg16_bn.pth"))
print(vgg16.classifier[6].out_features) # 1000 


# Freeze training for all layers
for param in vgg16.features.parameters():
    param.require_grad = False

# Newly created modules have require_grad=True by default
num_features = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1] # Remove last layer
features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 4 outputs
vgg16.classifier = nn.Sequential(*features) # Replace the model classifier
print(vgg16)

In [ ]:

# Load the pre-trained VGG19 model with batch normalization
vgg19 = models.vgg19_bn(pretrained=True)
vgg19.load_state_dict(torch.load("/kaggle/input/vgg19bn/vgg19_bn.pth"))
# Print the original number of output features in the classifier (usually 1000 for ImageNet)
print(vgg19.classifier[6].out_features)

# Freeze training for all layers
for param in vgg19.features.parameters():
    param.requires_grad = False

# Modify the classifier to match your task
num_features = vgg19.classifier[6].in_features
num_classes = len(class_names)  # Replace with the actual number of classes
classifier = nn.Sequential(
    nn.Linear(num_features, 512),  # Add a hidden layer if needed
    nn.ReLU(),  # Activation function
    nn.Dropout(0.5),  # Dropout for regularization
    nn.Linear(512, num_classes)  # Output layer with the desired number of classes
)

vgg19.classifier[6] = nn.Linear(num_features, num_classes)  # Replace the final layer

# Now, your VGG19 model is ready for fine-tuning with the modified classifier
print(vgg19)


In [ ]:
# Load the pretrained model from pytorch
# vgg19 not 16
vgg16 = models.vgg16_bn()
vgg16.load_state_dict(torch.load("/kaggle/input/vgg19bn/vgg19_bn.pth"))
print(vgg16.classifier[6].out_features) # 1000 


# Freeze training for all layers
for param in vgg16.features.parameters():
    param.require_grad = False

# Newly created modules have require_grad=True by default
num_features = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1] # Remove last layer
features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 4 outputs
vgg16.classifier = nn.Sequential(*features) # Replace the model classifier
print(vgg16)

In [ ]:
# If you want to train the model for more than 2 epochs, set this to True after the first run
resume_training = True

if resume_training:
    print("Loading pretrained model..")
    vgg16.load_state_dict(torch.load('/kaggle/input/vgg16-transfer-learning-pytorch/VGG16_v2-OCT_Retina_half_dataset.pt'))
    print("Loaded!")

In [ ]:
if use_gpu:
    vgg16.cuda() #.cuda() will move everything to the GPU side
    
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
if use_gpu:
    vgg19.cuda() #.cuda() will move everything to the GPU side
    
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# Before Training

In [ ]:
print("Test before training")
eval_model(vgg16, criterion)

In [ ]:
visualize_model(vgg16)

In [ ]:
import copy

In [ ]:

# def train_model(vgg, criterion, optimizer, scheduler, num_epochs=10):
#     since = time.time()
#     best_model_wts = copy.deepcopy(vgg.state_dict())
#     best_acc = 0.0
    
#     avg_loss = 0
#     avg_acc = 0
#     avg_loss_val = 0
#     avg_acc_val = 0
    
#     train_batches = len(dataloaders[TRAIN])
#     val_batches = len(dataloaders[VAL])
    
#     for epoch in range(num_epochs):
# #         loop = tqdm.notebook.tqdm(train_dataloader)
# #         loop.set_description(f"{e+1}/{EPOCHS}")
#         print("Epoch {}/{}".format(epoch, num_epochs))
#         print('-' * 10)
        
#         loss_train = 0
#         loss_val = 0
#         acc_train = 0
#         acc_val = 0
        
#         vgg.train(True)
        
#         for i, data in enumerate(dataloaders[TRAIN]):
#             if i % 100 == 0:
#                 print("\rTraining batch {}/{}".format(i, train_batches / 2), end='', flush=True)
                
#             # Use half training dataset
#             if i >= train_batches / 2:
#                 break
                
#             inputs, labels = data
#             inputs, labels = inputs.cuda(), labels.cuda()
            
#             optimizer.zero_grad()
            
#             outputs = vgg(inputs)
            
#             _, preds = torch.max(outputs.data, 1)
#             loss = criterion(outputs, labels)
            
#             loss.backward()
#             optimizer.step()
    
#             loss_train += loss.data
#             acc_train += torch.sum(preds == labels.data)
            
#             del inputs, labels, outputs, preds
#             torch.cuda.empty_cache()
        
#         print()
#         # * 2 as we only used half of the dataset
#         avg_loss = loss_train * 2 / dataset_sizes[TRAIN]
#         avg_acc = acc_train * 2 / dataset_sizes[TRAIN]
        
#         vgg.train(False)
#         vgg.eval()
            
#         for i, data in enumerate(dataloaders[VAL]):
#             if i % 100 == 0:
#                 print("\rValidation batch {}/{}".format(i, val_batches), end='', flush=True)
                
#             inputs, labels = data
#             inputs, labels = inputs.cuda(), labels.cuda()

#             optimizer.zero_grad()
            
#             outputs = vgg(inputs)
            
#             _, preds = torch.max(outputs.data, 1)
#             loss = criterion(outputs, labels)
            
#             loss_val += loss.data
#             acc_val += torch.sum(preds == labels.data)
            
#             del inputs, labels, outputs, preds
#             torch.cuda.empty_cache()
        
#         avg_loss_val = loss_val / dataset_sizes[VAL]
#         avg_acc_val = acc_val / dataset_sizes[VAL]
        
#         print()
#         print("Epoch {} result: ".format(epoch))
#         print("Avg loss (train): {:.4f}".format(avg_loss))
#         print("Avg acc (train): {:.4f}".format(avg_acc))
#         print("Avg loss (val): {:.4f}".format(avg_loss_val))
#         print("Avg acc (val): {:.4f}".format(avg_acc_val))
#         print('-' * 10)
#         print()
        
#         if avg_acc_val > best_acc:
#             best_acc = avg_acc_val
#             best_model_wts = copy.deepcopy(vgg.state_dict())
        
#     elapsed_time = time.time() - since
#     print()
#     print("Training completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
#     print("Best acc: {:.4f}".format(best_acc))
    
#     vgg.load_state_dict(best_model_wts)
#     return vgg


# Training

In [ ]:
from tqdm import tqdm

In [ ]:
train_loss = []
val_loss = []
train_acc = []
val_acc = []

In [ ]:
# testing
def train_model(vgg, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(vgg.state_dict())
    best_acc = 0.0
    
    avg_loss = 0
    avg_acc = 0
    avg_loss_val = 0
    avg_acc_val = 0
    
    train_batches = len(dataloaders[TRAIN])
    val_batches = len(dataloaders[VAL])
    
    for epoch in tqdm(range(num_epochs), desc = "Epoch"):
#         loop = tqdm.notebook.tqdm(train_dataloader)
#         loop.set_description(f"{e+1}/{EPOCHS}")
#         print("Epoch {}/{}".format(epoch, num_epochs))
#         print('-' * 10)
        
        loss_train = 0
        loss_val = 0
        acc_train = 0
        acc_val = 0
        
        vgg.train(True)
        
        for i, data in enumerate(dataloaders[TRAIN]):
            if i % 100 == 0:
                print("\rTraining batch {}/{}".format(i, train_batches / 2), end='', flush=True)
                
            # Use half training dataset
            if i >= train_batches / 2:
                break
                
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            optimizer.zero_grad()
            
            outputs = vgg(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
#             train_loss.append(loss.data)
            loss.backward()
            optimizer.step()
            
            loss_train += loss.data
            acc_train += torch.sum(preds == labels.data)
#             train_loss.append(loss_train)
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
        
        print()
        # * 2 as we only used half of the dataset
        avg_loss = loss_train * 2 / dataset_sizes[TRAIN]
        avg_acc = acc_train * 2 / dataset_sizes[TRAIN]
        train_loss.append(avg_loss)
        train_acc.append(avg_acc)
        vgg.train(False)
        vgg.eval()
            
        for i, data in enumerate(dataloaders[VAL]):
            if i % 100 == 0:
                print("\rValidation batch {}/{}".format(i, val_batches), end='', flush=True)
                
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()

            optimizer.zero_grad()
            
            outputs = vgg(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss_val += loss.data
            acc_val += torch.sum(preds == labels.data)
            
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
        
        avg_loss_val = loss_val / dataset_sizes[VAL]
        avg_acc_val = acc_val / dataset_sizes[VAL]
        val_loss.append(avg_loss_val)
        val_acc.append(avg_acc_val)
        
        print()
        print("Epoch {} result: ".format(epoch + 1))
        print("Avg loss (train): {:.4f}".format(avg_loss))
        print("Avg acc (train): {:.4f}".format(avg_acc))
        print("Avg loss (val): {:.4f}".format(avg_loss_val))
        print("Avg acc (val): {:.4f}".format(avg_acc_val))
        print('-' * 10)
        print()
        
        if avg_acc_val > best_acc:
            best_acc = avg_acc_val
            best_model_wts = copy.deepcopy(vgg.state_dict())
        
    elapsed_time = time.time() - since
    print('Completed.')
    print("Training completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Best acc: {:.4f}".format(best_acc))
    
    vgg.load_state_dict(best_model_wts)
    return vgg

In [ ]:
vgg16 = train_model(vgg16, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=5)

In [ ]:
print(train_loss)
print(val_loss) 
print(train_acc) 
print(val_acc) 

In [ ]:
vgg19 = train_model(vgg19, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=5)

In [ ]:
torch.save(vgg16.state_dict(), 'VGG16_potato-disease-leaf-dataset.pt')

In [ ]:
eval_model(vgg16, criterion)

In [ ]:
visualize_model(vgg16, num_images=32)

In [ ]:
len(val_loss)

In [ ]:
loss_train = []
acc_train = []
loss_val = []
acc_val = []

In [ ]:
def rechange(model_check):
    model_new = []
    for i in model_check:
        i = i.item()
        model_new.append(i)
    return model_new
loss_train = rechange(train_loss)
acc_train = rechange(train_acc)
loss_val = rechange(val_loss)
acc_val = rechange(val_acc)

In [ ]:
plt.plot(loss_train, label='train')
plt.plot(loss_val, label='val', color = 'red')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
plt.plot(acc_train, label='train')
plt.plot(acc_val, label='val', color = 'red')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
# loss_model.append(0.01)

In [ ]:
plt.plot(loss_model, label='val')
plt.title('Model Acc')
plt.ylabel('Acc')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
max(loss_model)

In [ ]:
plt.plot(accuracy, label='train')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend()
plt.show()